# Ciência de Dados 2

### Extracao de dados do Twitter

In [1]:
#importing libraries
import pandas as pd
import requests

%matplotlib inline

#bearer token para acesso no twitter
TOKEN = 'Bearer '

# api-endpoints
URL_USER_LOOKUP = "https://api.twitter.com/2/users/by?"
URL_USER_TIMELINE = "https://api.twitter.com/2/users" 

### Para comparacao de prefeituras utilizaremos o Twitter para extracao de 3200 tweets da timeline de cada prefeitura


##### baseado na documentacao do twitter : https://developer.twitter.com/en/docs/twitter-api/tweets/timelines/api-reference/get-users-id-tweets

### Para a construção do dataset das prefeituras utilizamos o portal covid19.io acessando-o no dia 10/04 e filtrando pelas top 100 cidades com o maior número de obitos acumulados e buscamos manualmente o perfil de cada prefeitura nos baseando no primeiro resultado para a pesquisa "twitter prefeitura NOME_DA_CIDADE". Checamos individualmente cada uma para ter certeza de que o perfil e oficial. 

#### fonte casos de COVID: https://brasil.io/covid19/



In [2]:
# lê o arquivo CSV
df = pd.read_csv('datasets/TwitterPrefeiturasCOVIDIO.csv', delimiter=";")

# mostra o conteúdo do DataFrame
df.head()

,Data,Municipio,UF,Confirmados,Confirmados 100k habitantes,Obitos,Letalidade,Obitos 100k habitantes,perfil
0,05/04/2021,São Paulo,SP,635.582,"5.156,75",22.844,"3,59%","185,34",prefsp
1,08/04/2021,Rio de Janeiro,RJ,235.005,"3.482,68",21.436,"9,12%","317,67",Prefeitura_Rio
2,08/04/2021,Manaus,AM,163.021,"7.344,68",8.520,"5,23%","383,86",PrefManaus
3,05/04/2021,Fortaleza,CE,171.154,"6.370,63",6.303,"3,68%","234,61",prefeiturapmf
4,08/04/2021,Brasília,DF,311.257,"10.187,95",6.066,"1,95%","198,55",BrasiliaDF


### Algmas das prefeituras não possuem perfil no twitter então vou removê-las

In [3]:
df[df['perfil'] == '-']

,Data,Municipio,UF,Confirmados,Confirmados 100k habitantes,Obitos,Letalidade,Obitos 100k habitantes,perfil
19,05/04/2021,Santo André,SP,44.154,"6.120,87",1.712,"3,88%","237,33",-
41,08/04/2021,Aparecida de Goiânia,GO,55.512,"9.406,49",1.019,"1,84%","172,67",-
55,21/03/2021,Contagem,MG,22.373,"3.344,50",818.000,"3,66%","122,28",-
67,05/04/2021,Piracicaba,SP,38.425,"9.435,19",673.000,"1,75%","165,25",-
69,05/04/2021,Guarujá,SP,13.389,"4.148,41",659.000,"4,92%","204,18",-
72,08/04/2021,Rondonópolis,MT,24.443,"10.355,36",649.000,"2,66%","274,95",-
77,04/04/2021,Feira de Santana,BA,34.710,"5.601,92",608.000,"1,75%","98,13",-
79,05/04/2021,Bauru,SP,30.146,"7.947,86",591.000,"1,96%","155,81",-
81,05/04/2021,Caucaia,CE,16.804,"4.601,16",583.000,"3,47%","159,63",-
87,08/04/2021,Teresópolis,RJ,20.828,"11.304,82",545.000,"2,62%","295,81",-


In [4]:
df = df[df['perfil'] != '-']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90 entries, 0 to 99
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Data                         90 non-null     object 
 1   Municipio                    90 non-null     object 
 2   UF                           90 non-null     object 
 3   Confirmados                  90 non-null     float64
 4   Confirmados 100k habitantes  90 non-null     object 
 5   Obitos                       90 non-null     float64
 6   Letalidade                   90 non-null     object 
 7   Obitos 100k habitantes       90 non-null     object 
 8   perfil                       90 non-null     object 
dtypes: float64(2), object(7)
memory usage: 7.0+ KB


### Antes de buscar os perfis no twitter precisamos dos IDs dos perfis para o download das timelines. Esta informação esta disponível na twitter API atraves do metodo Lookup, entao criamos uma nova coluna TwitterID

##### fonte: https://developer.twitter.com/en/docs/twitter-api/v1/accounts-and-users/follow-search-get-users/api-reference/get-users-lookup

In [5]:
df['TwitterID'] = 0
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90 entries, 0 to 99
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Data                         90 non-null     object 
 1   Municipio                    90 non-null     object 
 2   UF                           90 non-null     object 
 3   Confirmados                  90 non-null     float64
 4   Confirmados 100k habitantes  90 non-null     object 
 5   Obitos                       90 non-null     float64
 6   Letalidade                   90 non-null     object 
 7   Obitos 100k habitantes       90 non-null     object 
 8   perfil                       90 non-null     object 
 9   TwitterID                    90 non-null     int64  
dtypes: float64(2), int64(1), object(7)
memory usage: 7.7+ KB


In [6]:
def add_twitter_id (perfil):
    response = requests.get(url = URL_USER_LOOKUP, params = {'usernames' : perfil}, headers={'Authorization': TOKEN}, timeout=2) 
    resp_dict = response.json()
    print(resp_dict)
    first_value = resp_dict['data']
    return first_value[0]['id']

In [7]:
df['TwitterID'] = df.apply(lambda row : add_twitter_id(row['perfil']), axis = 1)


df

{'data': [{'id': '1319636844821217280', 'name': 'Cidade de São Paulo', 'username': 'prefsp'}]}
{'data': [{'id': '1897952138', 'name': 'Prefeitura do Rio', 'username': 'Prefeitura_Rio'}]}
{'data': [{'id': '801439438983270400', 'name': 'Prefeitura de Manaus', 'username': 'PrefManaus'}]}
{'data': [{'id': '771896068451758084', 'name': 'Prefeitura de Fortaleza', 'username': 'prefeiturapmf'}]}
{'data': [{'id': '24695159', 'name': 'Brasília DF', 'username': 'BrasiliaDF'}]}
{'data': [{'id': '69284025', 'name': 'PrefSalvador', 'username': 'PrefSalvador'}]}
{'data': [{'id': '97251131', 'name': 'Porto Alegre', 'username': 'Prefeitura_POA'}]}
{'data': [{'id': '1305480986512293889', 'name': 'Prefeitura de Goiânia', 'username': 'prefeituradegyn'}]}
{'data': [{'id': '1934139290', 'name': 'Prefeitura de Belém', 'username': 'prefeiturabelem'}]}
{'data': [{'id': '68693419', 'name': 'Prefeitura Curitiba #FiqueEmCasa', 'username': 'Curitiba_PMC'}]}
{'data': [{'id': '75342150', 'name': 'Prefeitura do Recif

{'data': [{'id': '2724530988', 'name': 'Prefeitura de Viamão', 'username': 'pmviamao'}]}
{'data': [{'id': '270941106', 'name': 'Prefeitura de Sobral', 'username': 'governodesobral'}]}
{'data': [{'id': '1249117465', 'name': 'Prefeitura Araçatuba', 'username': 'AracatubaPMA'}]}


,Data,Municipio,UF,Confirmados,Confirmados 100k habitantes,Obitos,Letalidade,Obitos 100k habitantes,perfil,TwitterID
0,05/04/2021,São Paulo,SP,635.582,"5.156,75",22.844,"3,59%","185,34",prefsp,1319636844821217280
1,08/04/2021,Rio de Janeiro,RJ,235.005,"3.482,68",21.436,"9,12%","317,67",Prefeitura_Rio,1897952138
2,08/04/2021,Manaus,AM,163.021,"7.344,68",8.520,"5,23%","383,86",PrefManaus,801439438983270400
3,05/04/2021,Fortaleza,CE,171.154,"6.370,63",6.303,"3,68%","234,61",prefeiturapmf,771896068451758084
4,08/04/2021,Brasília,DF,311.257,"10.187,95",6.066,"1,95%","198,55",BrasiliaDF,24695159
...,...,...,...,...,...,...,...,...,...,...
95,04/04/2021,Itabuna,BA,27.502,"12.870,35",490.000,"1,78%","229,31",itabunaba,52928395
96,08/04/2021,Caruaru,PE,18.799,"5.146,49",490.000,"2,61%","134,14",prefcaruaru,1334501712355332097
97,09/04/2021,Viamão,RS,6.754,"2.635,17",490.000,"7,25%","191,18",pmviamao,2724530988
98,05/04/2021,Sobral,CE,18.679,"8.864,75",481.000,"2,58%","228,27",governodesobral,270941106


In [8]:
## Salvando os IDs das prefeituras para nao precisar fazer de novo a cada execucao do notebook
df.to_csv('datasets/TwitterPrefsComID.csv', sep=';', encoding='utf-8')

### Extracao da timeline de cada prefeitura

#### Extraimos um .json com todas as informacoes referentes as prefeituras para futuras analises. 

In [206]:
import json

infoprefeituras = [];

In [223]:
def get_twitter_information (twitterid, municipio):
    url = URL_USER_TIMELINE + "/" + twitterid + "/tweets?"
    pagination_token = ''
    
    all_tweets=[]
    all_includes=[]
    
    params = {'tweet.fields' : 'created_at,public_metrics,referenced_tweets' , 'expansions' : 'author_id,referenced_tweets.id,entities.mentions.username,referenced_tweets.id.author_id' , 'user.fields' : 'created_at' ,'max_results' : '100'}
    
    while True:
        if pagination_token != '':
            params = {'tweet.fields' : 'created_at,entities,public_metrics,referenced_tweets' , 'expansions' : 'author_id,referenced_tweets.id,entities.mentions.username,referenced_tweets.id.author_id' , 'user.fields' : 'created_at' ,'max_results' : '100', 'pagination_token' : pagination_token}
            
        response = requests.get(url = url, params = params, headers={'Authorization': TOKEN}, timeout=2) 
        resp_dict = response.json()
        
        if 'meta' in resp_dict:
            meta_value = resp_dict['meta']

            if meta_value['result_count'] > 0 :
                tweet_values = resp_dict['data']
                includes_value = resp_dict['includes']

                all_tweets.append(tweet_values)
                all_includes.append(includes_value)

                if 'next_token' in meta_value:
                    pagination_token = meta_value['next_token']

                else:
                    print("Nao ha next_token - Acabou as paginas")
                    print("Params")
                    print(params)
                    print("Municipio")
                    print(municipio)
                    print("Mensagem da API")
                    print(meta_value)
                    pagination_token = ''
                    break;


            else:
                print("Nao ha mais tweets - Result count 0")
                print("Params")
                print(params)
                print("Municipio")
                print(municipio)
                print("Mensagem da API")
                print(resp_dict)
                pagination_token = ''
                break;
                
        else:
            print("Ocorreu um erro e a API nao retornou nenhum tweet")
            print("Params")
            print(params)
            print("Municipio")
            print(municipio)
            print("Mensagem da API")
            print(resp_dict)
            break;
    
    infoprefeituras.append({'municipio' : municipio, 'tweets' : all_tweets, 'includes' : all_includes})
    
    return True;         
    

In [208]:
df['Processado'] = df.apply(lambda row : get_twitter_information(row['TwitterID'], row['Municipio']), axis = 1)

## Salvando os resultados para consulta futura
df.to_csv('datasets/TwitterPrefsCOVIDComID.csv', sep=';', encoding='utf-8')

## Salvando o json file.

with open('PrefsTwitter.json', 'w') as jsonfile:
    json.dump(infoprefeituras, jsonfile)  

Nao ha next_token - Acabou as paginas
Params
{'tweet.fields': 'created_at,entities,public_metrics,referenced_tweets', 'expansions': 'author_id,referenced_tweets.id,entities.mentions.username,referenced_tweets.id.author_id', 'user.fields': 'created_at', 'max_results': '100', 'pagination_token': '7140dibdnow9c7btw3w29gqowa4ywr2huy85nrzslwgmu'}
Municipio
São Paulo
Mensagem da API
{'oldest_id': '1333409812378423298', 'newest_id': '1334180572374110208', 'result_count': 40, 'previous_token': '77qpymm88g5h9vqkluldpw91lqdm5qvs70vee16t8p7w8'}
Nao ha next_token - Acabou as paginas
Params
{'tweet.fields': 'created_at,entities,public_metrics,referenced_tweets', 'expansions': 'author_id,referenced_tweets.id,entities.mentions.username,referenced_tweets.id.author_id', 'user.fields': 'created_at', 'max_results': '100', 'pagination_token': '7140dibdnow9c7btw3w29aeat5me042ndvmgy4meihmig'}
Municipio
Rio de Janeiro
Mensagem da API
{'oldest_id': '1330628934476713985', 'newest_id': '1331673998485286913', 'r

Nao ha mais tweets - Result count 0
Params
{'tweet.fields': 'created_at,entities,public_metrics,referenced_tweets', 'expansions': 'author_id,referenced_tweets.id,entities.mentions.username,referenced_tweets.id.author_id', 'user.fields': 'created_at', 'max_results': '100', 'pagination_token': 'zldjwdz3w6sba13nhr56ml162e7kyeqp9egbxy6i1x4'}
Municipio
Porto Velho
Mensagem da API
{'meta': {'result_count': 0, 'previous_token': '10iy0xk016rrqp9xa8kkuviju43nvfsxldecy4tl3rqv'}}
Nao ha next_token - Acabou as paginas
Params
{'tweet.fields': 'created_at,entities,public_metrics,referenced_tweets', 'expansions': 'author_id,referenced_tweets.id,entities.mentions.username,referenced_tweets.id.author_id', 'user.fields': 'created_at', 'max_results': '100', 'pagination_token': '9557owtzx4m9b4ehltzvsrec2jtgf1x'}
Municipio
São Bernardo do Campo
Mensagem da API
{'oldest_id': '4472244398', 'newest_id': '4636542690', 'result_count': 17, 'previous_token': '9drmqzx45lch5m3jp1vb9wvulzqndbk'}
Nao ha next_token - 

Nao ha mais tweets - Result count 0
Params
{'tweet.fields': 'created_at,entities,public_metrics,referenced_tweets', 'expansions': 'author_id,referenced_tweets.id,entities.mentions.username,referenced_tweets.id.author_id', 'user.fields': 'created_at', 'max_results': '100', 'pagination_token': 'zldjwdz3w6sba13nhrkee5asap0khocyenyfv6bniux'}
Municipio
Jaboatão dos Guararapes
Mensagem da API
{'meta': {'result_count': 0, 'previous_token': '10iy0xk016rrqp9xa8l02n2tgceguz2juinv221q98oo'}}
Nao ha next_token - Acabou as paginas
Params
{'tweet.fields': 'created_at,entities,public_metrics,referenced_tweets', 'expansions': 'author_id,referenced_tweets.id,entities.mentions.username,referenced_tweets.id.author_id', 'user.fields': 'created_at', 'max_results': '100', 'pagination_token': '7140dibdnow9c7btw3n51eamoszbacqld2ps9csmd8ppu'}
Municipio
Duque de Caxias
Mensagem da API
{'oldest_id': '1074623626819895297', 'newest_id': '1080513736324964352', 'result_count': 28, 'previous_token': '77qpymm88g5h9vqk

KeyError: 'meta'

### Na linha 42 deu algum erro na extracao, porem para nao ultrapassar o limite de tweets por conta (500 000 - neste momento ja estou com 233 400) decido cortar o dataframe e continuar a parte da cidade 42 (Londrina)

### Update: O erro era que a prefeitura de Maua nao esta com o perfil publico (nao sei qual o sentido de ter uma conta de prefeitura no twitter sem ser aberta para o publico)

In [217]:
dfCidRestantes = df.iloc[42:90]
dfCidRestantes

,Data,Municipio,UF,Confirmados,Confirmados 100k habitantes,Obitos,Letalidade,Obitos 100k habitantes,perfil,TwitterID,Processado
44,05/04/2021,Mauá,SP,17.815,"3.730,48",949.0,"5,33%","198,72",PrefeituraMaua,127518831,True
45,08/04/2021,Serra,ES,49.410,"9.371,44",941.0,"1,90%","178,48",prefeituraserra,1306633006182957056,True
46,21/03/2021,Juiz de Fora,MG,22.278,"3.886,03",929.0,"4,17%","162,05",PrefeituraJF,1052984598,True
47,08/04/2021,Várzea Grande,MT,20.937,"7.281,78",927.0,"4,43%","322,41",prefeituravg,432461172,True
48,05/04/2021,Mogi das Cruzes,SP,19.646,"4.358,18",913.0,"4,65%","202,54",prefeituramogi,18560784,True
49,08/04/2021,Cariacica,ES,30.538,"7.954,32",909.0,"2,98%","236,77",prefcariacica,1310641080434651142,True
50,08/04/2021,Vitória,ES,43.288,"11.832,01",872.0,"2,01%","238,35",vitoriaonline,1315677319596515330,True
51,08/04/2021,Campos dos Goytacazes,RJ,20.680,"4.045,64",853.0,"4,12%","166,87",PrefCampos,94634629,True
52,08/04/2021,Florianópolis,SC,70.797,"13.913,79",852.0,"1,20%","167,44",scflorianopolis,92527636,True
53,08/04/2021,Maringá,PR,41.586,"9.667,63",846.0,"2,03%","196,67",prefeiturademga,1330855278452084738,True


In [225]:
dfCidRestantes['Processado'] = dfCidRestantes.apply(lambda row : get_twitter_information(row['TwitterID'], row['Municipio']), axis = 1)

## Salvando o json file.
with open('PrefsTwitter.json', 'w') as jsonfile:
    json.dump(infoprefeituras, jsonfile)

Ocorreu um erro e a API nao retornou nenhum tweet
Params
{'tweet.fields': 'created_at,entities,public_metrics,referenced_tweets', 'expansions': 'author_id,referenced_tweets.id,entities.mentions.username,referenced_tweets.id.author_id', 'user.fields': 'created_at', 'max_results': '100'}
Municipio
Mauá
Mensagem da API
{'errors': [{'resource_id': '127518831', 'parameter': 'id', 'resource_type': 'user', 'section': 'data', 'title': 'Authorization Error', 'value': '127518831', 'detail': 'Sorry, you are not authorized to see the user with id: [127518831].', 'type': 'https://api.twitter.com/2/problems/not-authorized-for-resource'}]}
Nao ha next_token - Acabou as paginas
Params
{'tweet.fields': 'created_at,entities,public_metrics,referenced_tweets', 'expansions': 'author_id,referenced_tweets.id,entities.mentions.username,referenced_tweets.id.author_id', 'user.fields': 'created_at', 'max_results': '100', 'pagination_token': '7140dibdnow9c7btw3w2ofy5dbzcmxpsjp07yhoo1viup'}
Municipio
Serra
Mensage

Nao ha mais tweets - Result count 0
Params
{'tweet.fields': 'created_at,entities,public_metrics,referenced_tweets', 'expansions': 'author_id,referenced_tweets.id,entities.mentions.username,referenced_tweets.id.author_id', 'user.fields': 'created_at', 'max_results': '100', 'pagination_token': 'zldjwdz3w6sba13n5sw8eicp48srojouj80k0bds2es'}
Municipio
Anápolis
Mensagem da API
{'meta': {'result_count': 0, 'previous_token': '10iy0xk016rrqp9x9wmbwnfvd5y925xvqn7x676sds8j'}}
Nao ha next_token - Acabou as paginas
Params
{'tweet.fields': 'created_at,entities,public_metrics,referenced_tweets', 'expansions': 'author_id,referenced_tweets.id,entities.mentions.username,referenced_tweets.id.author_id', 'user.fields': 'created_at', 'max_results': '100', 'pagination_token': 'zldjwdz3w6sba13nersszzbc9tyo8v6tdngicmjteeq'}
Municipio
São Vicente
Mensagem da API
{'oldest_id': '535387087462465536', 'newest_id': '536579945087373314', 'result_count': 19, 'previous_token': '10iy0xk016rrqp9xa5l8h8m96x1ggqbnaazu5zd

Nao ha mais tweets - Result count 0
Params
{'tweet.fields': 'created_at,entities,public_metrics,referenced_tweets', 'expansions': 'author_id,referenced_tweets.id,entities.mentions.username,referenced_tweets.id.author_id', 'user.fields': 'created_at', 'max_results': '100', 'pagination_token': 'zldjwdz3w6sba13n5s1x6mimm38i4avxlyemwhklf7c'}
Municipio
Ananindeua
Mensagem da API
{'meta': {'result_count': 0, 'previous_token': '10iy0xk016rrqp9x9wlhlfk1ansoslp2tpyb93cz7513'}}
Nao ha next_token - Acabou as paginas
Params
{'tweet.fields': 'created_at,entities,public_metrics,referenced_tweets', 'expansions': 'author_id,referenced_tweets.id,entities.mentions.username,referenced_tweets.id.author_id', 'user.fields': 'created_at', 'max_results': '100'}
Municipio
Alvorada
Mensagem da API
{'oldest_id': '2049215173758976', 'newest_id': '2049215173758976', 'result_count': 1}
Nao ha mais tweets - Result count 0
Params
{'tweet.fields': 'created_at,entities,public_metrics,referenced_tweets', 'expansions': '

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## Extracao dos dados realizada. Agora vamos limpar os dados para analisa-los